In [88]:
import requests
import json

import pandas as pd
from pandas.tseries.frequencies import to_offset

import numpy as np

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
import math

from sklearn.preprocessing import MinMaxScaler

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.callbacks import EarlyStopping

import time
import datetime
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.metrics import mean_absolute_percentage_error

from sklearn.ensemble import ExtraTreesClassifier
# from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
# from sklearn.tree import DecisionTreeRegressor
import warnings
warnings.filterwarnings('ignore')

In [89]:
data = pd.read_csv("../Dataset/STB.csv")
data.rename(columns={"trunc_time":"Date","open_price":"open","high_price":"high","low_price":"low","close_price":"Close"}, inplace= True)

# X = data.drop(['Date', 'Close'], axis=1).values
# y = data['Close'].values

# Split data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)



In [90]:
from sklearn.preprocessing import MinMaxScaler
closedf = data[['Date','Close']]
print(closedf)
print("Shape of close dataframe:", closedf.shape)
close_stock = closedf.copy()
del closedf['Date']
scaler=MinMaxScaler(feature_range=(0,1))
closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))
print(closedf.shape)
time_step = 20


            Date  Close
0     2016-01-04  12600
1     2016-01-05  12300
2     2016-01-06  12800
3     2016-01-07  12600
4     2016-01-08  12600
...          ...    ...
1722  2022-11-21  16700
1723  2022-11-22  16900
1724  2022-11-23  17500
1725  2022-11-24  18200
1726  2022-11-25  18900

[1727 rows x 2 columns]
Shape of close dataframe: (1727, 2)
(1727, 1)


In [91]:
print("start")
segment_size = 100
print(closedf)

# Calculate the number of segments in the dataset
num_segments = len(closedf) // segment_size
print(num_segments)

# Create a list to hold the moving average values
moving_averages = []

# Calculate the moving average for each segment
for i in range(num_segments):
    print("start")
    start = i * segment_size
    end = start + segment_size
    segment = closedf[start:end]
    a = []
    for i in segment.tolist():
        a += i
    print(a)
    moving_average = a.rolling(window=5).mean()
    print(moving_averages)

    moving_averages.extend(moving_average.tolist())

# Add the moving averages as a new feature in the dataset
closedf['moving_average'] = moving_averages
print("end")

start
[[0.18563923]
 [0.17513135]
 [0.19264448]
 ...
 [0.35726795]
 [0.38178634]
 [0.40630473]]
17
start
[0.1856392294220665, 0.1751313485113835, 0.19264448336252188, 0.1856392294220665, 0.1856392294220665, 0.1751313485113835, 0.18213660245183882, 0.1786339754816112, 0.16112084063047283, 0.13309982486865146, 0.11908931698774078, 0.13660245183887915, 0.1260945709281961, 0.1155866900175131, 0.1260945709281961, 0.15061295971978983, 0.13309982486865146, 0.13660245183887915, 0.14010507880910678, 0.14010507880910678, 0.13660245183887915, 0.13309982486865146, 0.13309982486865146, 0.13660245183887915, 0.13660245183887915, 0.12959719789842378, 0.1260945709281961, 0.1155866900175131, 0.12959719789842378, 0.12959719789842378, 0.12259194395796846, 0.1260945709281961, 0.13660245183887915, 0.13309982486865146, 0.13660245183887915, 0.12959719789842378, 0.1260945709281961, 0.13309982486865146, 0.12959719789842378, 0.12959719789842378, 0.12959719789842378, 0.1260945709281961, 0.1260945709281961, 0.1260

AttributeError: 'list' object has no attribute 'rolling'

In [ ]:
training_size=int(len(closedf)*0.8)
test_size=len(closedf)-training_size
train_data,test_data=closedf[0:training_size,:],closedf[training_size:len(closedf),:1]
print("train_data: ", train_data.shape)
print("test_data: ", test_data.shape)
train_data

train_data:  (1381, 1)
test_data:  (346, 1)


array([[0.18563923],
       [0.17513135],
       [0.19264448],
       ...,
       [0.80210158],
       [0.76357268],
       [0.73905429]])

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 10
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

print("X_train: ", X_train)
print("y_train: ", y_train)
print("X_test: ", X_test.shape)
print("y_test", y_test.shape)

X_train:  [[0.18563923 0.17513135 0.19264448 ... 0.17863398 0.16112084 0.13309982]
 [0.17513135 0.19264448 0.18563923 ... 0.16112084 0.13309982 0.11908932]
 [0.19264448 0.18563923 0.18563923 ... 0.13309982 0.11908932 0.13660245]
 ...
 [0.78633975 0.76007005 0.77758319 ... 0.84763573 0.88616462 0.80735552]
 [0.76007005 0.77758319 0.80210158 ... 0.88616462 0.80735552 0.83187391]
 [0.77758319 0.80210158 0.83012259 ... 0.80735552 0.83187391 0.80210158]]
y_train:  [0.11908932 0.13660245 0.12609457 ... 0.83187391 0.80210158 0.76357268]
X_test:  (335, 10)
y_test (335,)


In [ ]:
class Node:
    def __init__(self, split_feature=None, split_value=None, left=None, right=None, prediction=None, is_leaf=False):
        self.split_feature = split_feature
        self.split_value = split_value
        self.left = left
        self.right = right
        self.prediction = prediction
        self.is_leaf = is_leaf


In [ ]:
import numpy as np

class DecisionTreeRegressor:
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1, max_features=None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y, depth=0)

    def _split(self, X, y):
        best_feature = None
        best_value = None
        best_score = float('inf')

        m, p = X.shape

        for feature in range(p):
            values = np.unique(X[:, feature])

            for value in values:
                left_indices = X[:, feature] <= value
                right_indices = X[:, feature] > value

                if len(y[left_indices]) < self.min_samples_leaf or len(y[right_indices]) < self.min_samples_leaf:
                    continue

                score = len(y[left_indices]) * np.var(y[left_indices]) + len(y[right_indices]) * np.var(y[right_indices])

                if score < best_score:
                    best_feature = feature
                    best_value = value
                    best_score = score

        return best_feature, best_value, best_score

    def _build_tree(self, X, y, node=None, depth=0):
        if node is None:
            node = Node()

        if depth == self.max_depth or len(y) < self.min_samples_split:
            node.is_leaf = True
            node.prediction = np.mean(y)
            return node

        split_feature, split_value, split_score = self._split(X, y)

        if split_feature is None:
            node.is_leaf = True
            node.prediction = np.mean(y)
            return node

        left_indices = X[:, split_feature] <= split_value
        right_indices = X[:, split_feature] > split_value

        node.split_feature = split_feature
        node.split_value = split_value

        node.left = self._build_tree(X[left_indices], y[left_indices], Node(), depth + 1)
        node.right = self._build_tree(X[right_indices], y[right_indices], Node(), depth + 1)

        return node
    def predict(self, X):
        return np.array([self._traverse_tree(x, self.tree) for x in X])
    
    def _traverse_tree(self, x, node):
        if node.is_leaf:
            return node.prediction
        else:
            if x[node.split_feature] <= node.split_value:
                return self._traverse_tree(x, node.left)
            else:
                return self._traverse_tree(x, node.right)

       


In [ ]:
import numpy as np

class RandomForestRegressor:
    def __init__(self, n_estimators=10, max_depth=None, min_samples_split=2, min_samples_leaf=1, max_features=None):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.trees = []
        
    def fit(self, X, y):
        for i in range(self.n_estimators):
            bootstrap_indices = np.random.choice(len(X), size=len(X), replace=True)
            bootstrap_X = X[bootstrap_indices]
            bootstrap_y = y[bootstrap_indices]
            
            tree = DecisionTreeRegressor(max_depth=self.max_depth, min_samples_split=self.min_samples_split, 
                                          min_samples_leaf=self.min_samples_leaf, max_features=self.max_features)
            tree.fit(bootstrap_X, bootstrap_y)
            self.trees.append(tree)
            
    def predict(self, X):
        return np.mean([tree.predict(X) for tree in self.trees], axis=0)


In [ ]:
from sklearn.metrics import mean_squared_error
import optuna
from sklearn.ensemble import RandomForestRegressor as SklearnRandom

# Define the objective function for Optuna
# def objective(trial):
#     n_estimators = trial.suggest_int('n_estimators', 10, 1000)
#     max_depth = trial.suggest_int('max_depth', 2, 32)
#     min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
#     min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
#     max_features = trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2'])
#     print(n_estimators)

#     model = RandomForestRegressor(
#         n_estimators=n_estimators,
#         max_depth=max_depth,
#         min_samples_split=min_samples_split,
#         min_samples_leaf=min_samples_leaf,
#         max_features=max_features    
#     )

#     print(n_estimators)
#     model.fit(X_train, y_train)
#     y_pred = model.predict(X_test)
#     mse = mean_squared_error(y_test, y_pred)

#     return mse

# # Use Optuna to optimize hyperparameters
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=2)
# best_params = study.best_params
# print("Optuna: ", best_params)

In [ ]:
# rf = RandomForestRegressor(**best_params)
# rf.fit(X_train, y_train)

In [ ]:
rf = SklearnRandom(max_depth=26, min_samples_leaf=2, min_samples_split=10,
                      n_estimators=160)
rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=26, min_samples_leaf=2, min_samples_split=10,
                      n_estimators=160)

In [ ]:
# Lets Do the prediction 

RF_train_predict=rf.predict(X_train)
RF_test_predict=rf.predict(X_test)
# print("Train data prediction:", train_predict)
# # print("Test data prediction:", test_predict)
RF_train_predict = RF_train_predict.reshape(-1,1)
RF_test_predict = RF_test_predict.reshape(-1,1)

print("Train data prediction:", RF_train_predict.shape)
print("Test data prediction:", RF_test_predict.shape)

Train data prediction: (1370, 1)
Test data prediction: (335, 1)


In [ ]:
# Transform back to original form

RF_train_predict = scaler.inverse_transform(RF_train_predict)
RF_test_predict = scaler.inverse_transform(RF_test_predict)
RF_original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
RF_original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

In [ ]:
# Evaluation metrices RMSE and MAE
RF_RMSE_train = math.sqrt(mean_squared_error(RF_original_ytrain,RF_train_predict))
RF_MSE_train = mean_squared_error(RF_original_ytrain,RF_train_predict)
RF_MAE_train = mean_absolute_error(RF_original_ytrain,RF_train_predict)

RF_RMSE_test = math.sqrt(mean_squared_error(RF_original_ytest,RF_test_predict))
RF_MSE_test = mean_squared_error(RF_original_ytest,RF_test_predict)
RF_MAE_test = mean_absolute_error(RF_original_ytest,RF_test_predict)

print("Train data RMSE: ", RF_RMSE_train)
print("Train data MSE: ", RF_MSE_train)
print("Test data MAE: ", RF_MAE_train)
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", RF_RMSE_test)
print("Test data MSE: ", RF_MSE_test)
print("Test data MAE: ", RF_MAE_test)

Train data RMSE:  228.03234074645948
Train data MSE:  51998.7484263094
Test data MAE:  144.35949398526415
-------------------------------------------------------------------------------------
Test data RMSE:  1749.5404100466405
Test data MSE:  3060891.646386167
Test data MAE:  1369.6776103554657


In [ ]:
RF_EV_train = explained_variance_score(RF_original_ytrain, RF_train_predict)
RF_EV_test = explained_variance_score(RF_original_ytest, RF_test_predict)

print("Train data explained variance regression score:", RF_EV_train)
print("Test data explained variance regression score:", RF_EV_test)

Train data explained variance regression score: 0.9969258520049741
Test data explained variance regression score: 0.8979157238780056


In [ ]:
RF_r2_train = r2_score(RF_original_ytrain, RF_train_predict)
RF_r2_test = r2_score(RF_original_ytest, RF_test_predict)

print("Train data R2 score:", RF_r2_train)
print("Test data R2 score:", RF_r2_test)

Train data R2 score: 0.9969258519621983
Test data R2 score: 0.8851232552707781


In [ ]:
RF_MGD_train = mean_gamma_deviance(RF_original_ytrain, RF_train_predict)
RF_MGD_test = mean_gamma_deviance(RF_original_ytest, RF_test_predict)
RF_MPD_train = mean_poisson_deviance(RF_original_ytrain, RF_train_predict)
RF_MPD_test = mean_poisson_deviance(RF_original_ytest, RF_test_predict)
print("Train data MGD: ", RF_MGD_train)
print("Test data MGD: ", RF_MGD_test)
print("----------------------------------------------------------------------")
print("Train data MPD: ", RF_MPD_train)
print("Test data MPD: ",RF_MPD_test)

Train data MGD:  0.00024118844552075116
Test data MGD:  0.004238735023477698
----------------------------------------------------------------------
Train data MPD:  3.3039412936183146
Test data MPD:  112.65492835837757


In [ ]:
# shift train predictions for plotting
from itertools import cycle
import plotly.express as px
look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(RF_train_predict)+look_back, :] = RF_train_predict
print("Train predicted data: ", trainPredictPlot.shape)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(RF_train_predict)+(look_back*2)+1:len(closedf)-1, :] = RF_test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])


plotdf = pd.DataFrame({'date': close_stock['Date'],
                       'original_close': close_stock['Close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Train predicted data:  (1727, 1)
Test predicted data:  (1727, 1)


In [ ]:
x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = 10
while(i<pred_days):
    
    if(len(temp_input)>time_step):
        
        x_input=np.array(temp_input[1:])
#         print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        
        yhat = rf.predict(x_input)
#         print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat.tolist())
        temp_input=temp_input[1:]
       
        lst_output.extend(yhat.tolist())
        i=i+1
        
    else:
        yhat = rf.predict(x_input)
        
        temp_input.extend(yhat.tolist())
        lst_output.extend(yhat.tolist())
        
        i=i+1
        
print("Output of predicted next days: ", len(lst_output))

Output of predicted next days:  10


In [ ]:
last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
print(last_days)
print(day_pred)

[ 1  2  3  4  5  6  7  8  9 10]
[11 12 13 14 15 16 17 18 19 20]


In [ ]:
import copy

temp_mat = np.empty((len(last_days)+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_days_value = copy.deepcopy(temp_mat)
next_predicted_days_value = copy.deepcopy(temp_mat)

last_original_days_value[0:time_step+1] = scaler.inverse_transform(closedf[len(closedf)-time_step:]).reshape(1,-1).tolist()[0]
next_predicted_days_value[time_step+1:] = scaler.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]
last_original_days_value.append(None)
names = cycle(['Last 15 days close price','Predicted next 10 days close price'])

new_pred_plot = pd.DataFrame({
    'last_original_days_value':last_original_days_value,
    'next_predicted_days_value':next_predicted_days_value
})

fig = px.line(new_pred_plot,x=new_pred_plot.index, y=[new_pred_plot['last_original_days_value'],
                                                      new_pred_plot['next_predicted_days_value']],
              labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Compare last 15 days vs next 10 days',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()